#### インポート

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import datetime
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from urllib.request import urlopen
import optuna.integration.lightgbm as lgb_o
from scipy.special import comb

#### クラス定義

In [2]:
# 訓練データと出馬表データを加工する抽象クラス
# 追加有
class DataProcessor:
    def __init__(self):
        self.data = pd.DataFrame()
        self.data_p = pd.DataFrame()  # preprocessing後の変数
        self.data_h = pd.DataFrame()  # horse_resultsをmergeした後の変数
        self.data_pe = pd.DataFrame()  # pedsをmergeした後の変数
        self.data_c = pd.DataFrame()  # カテゴリ変数化した後の変数
        
    # 馬の過去成績データの追加
    # 追加有
    def merge_horse_results(self, hr, n_samples_list=[5, 9, 'all']):
        self.data_h = self.data_p.copy()
        for n_samples in n_samples_list:
            self.data_h = hr.merge_all(self.data_h, n_samples=n_samples)
        # 4/6馬の出走間隔追加
        self.data_h['interval'] = (self.data_h['date'] - self.data_h['latest']).dt.days
        self.data_h.drop(['開催', 'latest'], axis=1, inplace=True)
            
    # 血統データ追加
    def merge_peds(self, peds):
        self.data_pe = self.data_h.merge(peds, left_on='horse_id', right_index=True, how='left')
        self.no_peds = self.data_pe[self.data_pe['peds_0'].isnull()]['horse_id'].unique()
        if len(self.no_peds):
            print('scrape peds at horse_id_list "no_peds"')
    
    # カテゴリ変数の処理
    def process_categorical(self, le_horse, le_jockey, results_m):
        df = self.data_pe.copy()
        
        # ラベルエンコーディング(horse_id, jockey_idを0始まりの整数に変換)
        # horse_id
        mask_horse = df['horse_id'].isin(le_horse.classes_)
        new_horse_id = df['horse_id'].mask(mask_horse).dropna().unique()
        le_horse.classes_ = np.concatenate([le_horse.classes_, new_horse_id])
        df['horse_id'] = le_horse.transform(df['horse_id'])
        
        # jockey_id
        mask_jockey = df['jockey_id'].isin(le_jockey.classes_)
        new_jockey_id = df['jockey_id'].mask(mask_jockey).dropna().unique()
        le_jockey.classes_ = np.concatenate([le_jockey.classes_, new_jockey_id])
        df['jockey_id'] = le_jockey.transform(df['jockey_id'])
        
        # horse_id, jockey_idをcategory型に変換
        df['horse_id'] = df['horse_id'].astype('category')
        df['jockey_id'] = df['jockey_id'].astype('category')
        
        # そのほかのカテゴリ変数をcategory型に変換してからダミー変数化
        weathers = results_m['weather'].unique()
        race_types = results_m['race_type'].unique()
        ground_states = results_m['ground_state'].unique()
        sexes = results_m['性'].unique()
        df['weather'] = pd.Categorical(df['weather'], weathers)
        df['race_type'] = pd.Categorical(df['race_type'], race_types)
        df['ground_state'] = pd.Categorical(df['ground_state'], ground_states)
        df['性'] = pd.Categorical(df['性'], sexes)
        
        df = pd.get_dummies(df, columns=['weather', 'race_type', 'ground_state', '性'])  
        self.data_c = df

# 予測に使う出馬表データを加工するクラス
# 追加有
class ShutubaTable(DataProcessor):
    def __init__(self, shutuba_tables):
        super(ShutubaTable, self).__init__()
        self.data = shutuba_tables
    
    # 出馬表データをスクレイピング
    # 追加有
    @classmethod
    def scrape(cls, race_id_list, date):
        data = pd.DataFrame()
        for race_id in tqdm(race_id_list):
            url = f'https://race.netkeiba.com/race/shutuba.html?race_id={race_id}'
            df = pd.read_html(url)[0]
            df = df.T.reset_index(level=0, drop=True).T

            html = requests.get(url)
            html.encoding = 'EUC-JP'
            soup = BeautifulSoup(html.text, 'html.parser')

            texts = soup.find('div', attrs={'class': 'RaceData01'}).text
            texts = re.findall(r'\w+', texts)
            for text in texts:
                # 4/6 -1に修正
                if 'm' in text:
                    df['course_len'] = [int(re.findall(r'\d+', text)[-1])] * len(df)
                if text in ['曇', '晴', '雨', '小雨', '小雪', '雪']:
                    df['weather'] = [text] * len(df)
                if text in ['良', '稍重', '重']:
                    df['ground_state'] = [text] * len(df)
                if '不' in text:
                    df['ground_state'] = ['不良'] * len(df)
                # 4/6追加
                if '稍' in text:
                    df['ground_state'] = ['稍重'] * len(df)
                if '芝' in text:
                    df['race_type'] = ['芝'] * len(df)
                if '障' in text:
                    df['race_type'] = ['障害'] * len(df)
                if 'ダ' in text:
                    df['race_type'] = ['ダート'] * len(df)
            df['date'] = [date] * len(df)

            # horse_id
            horse_id_list = []
            horse_td_list = soup.find_all('td', attrs={'class': 'HorseInfo'})
            for td in horse_td_list:
                horse_id = re.findall(r'\d+', td.find('a')['href'])[0]
                horse_id_list.append(horse_id)
            df['horse_id'] = horse_id_list

            # jockey_id
            jockey_id_list = []
            jockey_td_list = soup.find_all('td', attrs={'class': 'Jockey'})
            for td in jockey_td_list:
                jockey_id = re.findall(r'\d+', td.find('a')['href'])[0]
                jockey_id_list.append(jockey_id)
            df['jockey_id'] = jockey_id_list

            df.index = [race_id] * len(df)
            data = data.append(df)
            time.sleep(1)
        
        return cls(data)
    
    # 前処理
    # 追加有
    def preprocessing(self):
        df = self.data.copy()
        
        # 性齢を性と年齢に分ける
        df['性'] = df['性齢'].map(lambda x: str(x)[0])
        df['年齢'] = df['性齢'].map(lambda x: str(x)[1:]).astype(int)

        # 馬体重を体重と体重変化に分ける
        df = df[df['馬体重(増減)'] != '--']
        df['体重'] = df['馬体重(増減)'].str.split('(', expand=True)[0].astype(int)
        df['体重変化'] = df['馬体重(増減)'].str.split('(', expand=True)[1].str[:-1].astype(int)
        # 4/6追加：増減が「前計不」などのときは欠損値にする
        df['体重変化'] = pd.to_numeric(df['体重変化'], errors='coerce')
        
        # 日付をdatetime型に変更
        df['date'] = pd.to_datetime(df['date'])

        # データをint, floatに変換
        df['枠'] = df['枠'].astype(int)
        df['馬番'] = df['馬番'].astype(int)
        df['斤量'] = df['斤量'].astype(int)
        
        # 4/6追加：開催場所
        df['開催'] = df.index.map(lambda x: str(x)[4:6])
        
        # 4/6追加：距離は10の位を切り捨てる
        df['course_len'] = df['course_len'].astype(float) // 100
        
        # 4/6追加：出走数
        df['n_horses'] = df.index.map(df.index.value_counts())
    
        # 必要な列だけにする
        df = df[['枠', '馬番', '斤量', 'course_len', 'weather', 'race_type', 'ground_state', 'date', 'horse_id', 'jockey_id', '性', '年齢', '体重', '体重変化', '開催', 'n_horses']]
        df['開催'] = df.index.map(lambda x:str(x)[4:6])

        self.data_p = df.rename(columns={'枠': '枠番'})

# 訓練に使う過去レースデータを加工するクラス
# 追加有
class Results(DataProcessor):
    def __init__(self, results):
        super(Results, self).__init__()
        self.data = results
    
    # pickleファイルの読み込みと結合
    @classmethod
    def read_pickle(cls, path_list):
        df = pd.read_pickle(path_list[0])
        for path in path_list[1:]:
            df = update_data(df, pd.read_pickle(path))
        return cls(df)
    
    # レース結果データをスクレイピング
    # 追加有
    @staticmethod
    def scrape(race_id_list):
        # race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
            try:
                url = f'https://db.netkeiba.com/race/{race_id}'
                # メインとなるテーブルデータを取得
                df = pd.read_html(url)[0]

                html = requests.get(url)
                html.encoding = 'EUC-JP'
                soup = BeautifulSoup(html.text, 'html.parser')

                # 天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
                texts = (
                    soup.find('div', attrs={'class': 'data_intro'}).find_all('p')[0].text
                    + soup.find('div', attrs={'class': 'data_intro'}).find_all('p')[1].text
                )
                info = re.findall(r'\w+', texts)
                for text in info:
                    if text in ['芝', 'ダート']:
                        df['race_type'] = [text] * len(df)
                    if '障' in text:
                        df['race_type'] = ['障害'] * len(df)
                    # 4/6 -1に修正
                    if 'm' in text:
                        df['course_len'] = [int(re.findall(r'\d+', text)[-1])] * len(df)
                    if text in ['良', '稍重', '重', '不良']:
                        df['ground_state'] = [text] * len(df)
                    if text in ['曇', '晴', '雨', '小雨', '小雪', '雪']:
                        df['weather'] = [text] * len(df)
                    if '年' in text:
                        df['date'] = [text] * len(df)

                # 馬ID、騎手IDをスクレイピング
                horse_id_list = []
                horse_a_list = soup.find('table', attrs={'summary': 'レース結果'}).find_all('a', attrs={'href': re.compile('^/horse')})
                for a in horse_a_list:
                    horse_id = re.findall(r'\d+', a['href'])
                    horse_id_list.append(horse_id[0])
                jockey_id_list = []
                jockey_a_list = soup.find("table", attrs={'summary': 'レース結果'}).find_all('a', attrs={'href': re.compile('^/jockey')})
                for a in jockey_a_list:
                    jockey_id = re.findall(r'\d+', a['href'])
                    jockey_id_list.append(jockey_id[0])
                df['horse_id'] = horse_id_list
                df['jockey_id'] = jockey_id_list

                #インデックスをrace_idにする
                df.index = [race_id] * len(df)

                race_results[race_id] = df
                time.sleep(1)
            # 存在しないrace_idを飛ばす
            except IndexError:
                continue
            # 存在しないrace_idでAttributeErrorになるページの対処
            except AttributeError:
                continue
            # wifiの接続が切れた時などでも途中までのデータを返せるようにする
            except Exception as e:
                print(e)
                break
            # Jupyterで停止ボタンを押した時の対処
            except:
                break

        # pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])

        return race_results_df
    
    # 前処理
    # 追加有
    def preprocessing(self):
        df = self.data.copy()

        # 着順に数字以外の文字列が含まれているものを取り除く
        df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
        df.dropna(subset=['着順'], inplace=True)
        df['着順'] = df['着順'].astype(int)
        df['rank'] = df['着順'].map(lambda x: 1 if x < 4 else 0)

        # 性齢を性と年齢に分ける
        df['性'] = df['性齢'].map(lambda x: str(x)[0])
        df['年齢'] = df['性齢'].map(lambda x: str(x)[1:]).astype(int)

        # 馬体重を体重と体重変化に分ける
        df['体重'] = df['馬体重'].str.split('(', expand=True)[0].astype(int)
        df['体重変化'] = df['馬体重'].str.split('(', expand=True)[1].str[:-1].astype(int)

        # データをint, floatに変換
        df['単勝'] = df['単勝'].astype(float)
        # 距離は10の位を切り捨てる
        df['course_len'] = df['course_len'].astype(float) // 100

        # 不要な列を削除
        df.drop(['タイム', '着差', '調教師', '性齢', '馬体重', '馬名', '騎手', '人気', '着順'], axis=1, inplace=True)

        df['date'] = pd.to_datetime(df['date'], format='%Y年%m月%d日')
        
        # 開催場所
        df['開催'] = df.index.map(lambda x: str(x)[4:6])
        
        # 6/6出走数追加
        df['n_horses'] = df.index.map(df.index.value_counts())

        self.data_p = df
    
    # カテゴリ変数の処理
    def process_categorical(self):
        self.le_horse = LabelEncoder().fit(self.data_pe['horse_id'])
        self.le_jockey = LabelEncoder().fit(self.data_pe['jockey_id'])        
        super().process_categorical(self.le_horse, self.le_jockey, self.data_pe)

# 馬の過去成績データを処理するクラス
# 追加有
class HorseResults:
    def __init__(self, horse_results):
        self.horse_results = horse_results[['日付', '着順', '賞金', '着差', '通過', '開催', '距離']]
        self.preprocessing()
    
    # pickleファイルの読み込みと結合
    @classmethod
    def read_pickle(cls, path_list):
        df = pd.read_pickle(path_list[0])
        for path in path_list[1:]:
            df = update_data(df, pd.read_pickle(path))
        return cls(df)
    
    # 馬の過去成績データをスクレイピング
    @staticmethod
    def scrape(horse_id_list):
        # horse_idをkeyにしてDataFrame型を格納
        horse_results = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = f'https://db.netkeiba.com/horse/{horse_id}'
                # メインとなるテーブルデータを取得
                df = pd.read_html(url)[3]
                # 受賞歴がある馬の場合、4番目に受賞歴テーブルが来るため、5番目のデータを取得する
                if df.columns[0] == '受賞歴':
                    df = pd.read_html(url)[4]
                df.index = [horse_id] * len(df)
                horse_results[horse_id] = df
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
        
        # pd.DataFrame型にして一つのデータにまとめる
        horse_results_df = pd.concat([horse_results[key] for key in horse_results])
        
        return horse_results_df
    
    # 前処理
    def preprocessing(self):
        df = self.horse_results.copy()
        
        # 着順に数字以外の文字列が含まれているものを取り除く
        df['着順'] = pd.to_numeric(df['着順'], errors='coerce')
        df.dropna(subset=['着順'], inplace=True)
        df['着順'] = df['着順'].astype(int)
        
        df['date'] = pd.to_datetime(df['日付'])
        df.drop(['日付'], axis=1, inplace=True)
        
        # 賞金の欠損値を0で埋める
        df['賞金'].fillna(0, inplace=True)
        
        # 1着の着差を0にする
        df['着差'] = df['着差'].map(lambda x: 0 if x < 0 else x)
                                            
        # レース展開データ
        def corner(x, n):
            if type(x) != str:
                return x
            # n=1: 最初のコーナー位置、n=4: 最終コーナー位置
            if n == 1:
                return int(re.findall(r'\d+', x)[0])
            if n == 4:
                return int(re.findall(r'\d+', x)[-1])
                                            
        df['first_corner'] = df['通過'].map(lambda x: corner(x, 1))
        df['final_corner'] = df['通過'].map(lambda x: corner(x, 4))
        
        df['final_to_rank'] = df['final_corner'] - df['着順']
        df['first_to_rank'] = df['first_corner'] - df['着順']
        df['first_to_final'] = df['first_corner'] - df['final_corner']
        
        # 開催場所
        df['開催'] = df['開催'].str.extract(r'(\D+)')[0].map(place_dict).fillna('11')
        # race_type
        df['race_type'] = df['距離'].str.extract(r'(\D+)')[0].map(race_type_dict)
        # 距離は10の位を切り捨てる
        df['course_len'] = df['距離'].str.extract(r'(\d+)').astype(int) // 100
        df.drop(['距離'], axis=1, inplace=True)
        
        # インデックス名を与える
        df.index.name = 'horse_id'
        
        self.horse_results = df
        self.target_list = ['着順', '賞金', '着差', 'first_corner', 'final_corner', 'final_to_rank', 'first_to_rank', 'first_to_final']
    
    # n_samplesレース分馬ごとに平均する
    # 追加有
    def average(self, horse_id_list, date, n_samples='all'):
        target_df = self.horse_results.query('index in @horse_id_list')
        
        # 過去何走分取り出すか指定
        if n_samples == 'all':
            filtered_df = target_df[target_df['date'] < date]
        elif n_samples > 0:
            filtered_df = target_df[target_df['date'] < date].sort_values('date', ascending=False).groupby(level=0).head(n_samples)
        else:
            raise Exception('n_samples must be integer')
        
        # 集計して辞書型に入れる
        self.average_dict = {}
        self.average_dict['non_category'] = filtered_df.groupby(level=0)[self.target_list].mean().add_suffix(f'_{n_samples}R')
        for column in ['course_len', 'race_type', '開催']:
            self.average_dict[column] = filtered_df.groupby(['horse_id', column])[self.target_list].mean().add_suffix(f'_{column}_{n_samples}R')
        
        # 4/6追加：全レースの日付を変数latestに格納
        if n_samples == 5:
            self.latest = filtered_df.groupby('horse_id')['date'].max().rename('latest')
    
    # 追加有
    def merge(self, results, date, n_samples='all'):
        df = results[results['date'] == date]
        horse_id_list = df['horse_id']
        self.average(horse_id_list, date, n_samples)
        merged_df = df.merge(self.average_dict['non_category'], left_on='horse_id', right_index=True, how='left')
        for column in ['course_len', 'race_type', '開催']:
            merged_df = merged_df.merge(self.average_dict[column], left_on=['horse_id', column], right_index=True, how='left')
            
        # 4/6追加：全レースの日付を変数latestに格納
        if n_samples == 5:
            merged_df = merged_df.merge(self.latest, left_on='horse_id', right_index=True, how='left')
            
        return merged_df
    
    def merge_all(self, results, n_samples='all'):
        date_list = results['date'].unique()
        merged_df = pd.concat([self.merge(results, date, n_samples) for date in tpdm(date_list)])
        return merged_df

# 血統データを処理するクラス
class Peds:
    def __init__(self, peds):
        self.peds = peds
        self.peds_e = pd.DataFrame()  # LabelEncodingしてcategory型にした変数
    
    # pickleファイルの読み込みと結合
    @classmethod
    def read_pickle(cls, path_list):
        df = pd.read_pickle(path_list[0])
        for path in path_list[1:]:
            df = update_data(df, pd.read_pickle(path))
        return cls(df)
    
    # 血糖データをスクレイピング
    @staticmethod
    def scrape(horse_id_list):
        peds_dict = {}
        for horse_id in tqdm(horse_id_list):
            try:
                url = f'https://db.netkeiba.com/horse/ped/{horse_id}'
                df = pd.read_html(url)[0]

                # 重複を削除して1列のSeries型データに治す
                generations = {}
                for i in reversed(range(5)):
                    generations[i] = df[i]
                    df.drop([i], axis=1, inplace=True)
                    df = df.drop_duplicates()

                ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
                peds_dict[horse_id] = ped.reset_index(drop=True)
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
        
        # 列名をpeds_0, ..., peds_61にする
        peds_df = pd.concat([peds_dict[key] for key in peds_dict], axis=1).T.add_prefix('peds_')
        
        return peds_df
    
    def encode(self):
        df = self.peds.copy()
        for column in df.columns:
            df[column] = LabelEncoder().fit_transform(df[column].fillna('Na'))
        self.peds_e = df.astype('category')

# 払い戻し表データを加工するクラス
class Return:
    def __init__(self, return_tables):
        self.return_tables = return_tables
    
    # pickleファイルの読み込みと結合
    @classmethod
    def read_pickle(cls, path_list):
        df = pd.read_pickle(path_list[0])
        for path in path_list[1:]:
            df = update_data(df, pd.read_pickle(path))
        return cls(df)
    
    # 払い戻し表データをスクレイピング
    @staticmethod
    def scrape(race_id_list):
        return_tables = {}
        for race_id in tqdm(race_id_list):
            try:
                url = f'https://db.netkeiba.com/race/{race_id}'

                # 複勝やワイドなどが区切られてしまうため、改行コードを文字列に変換して後でsplitする
                f = urlopen(url)
                html = f.read()
                html = html.replace(b'<br />', b'br')            
                dfs = pd.read_html(html)

                # dfsの2番目に単勝～馬連、3番目にワイド～3連単がある
                df = pd.concat([dfs[1], dfs[2]])
                
                df.index = [race_id] * len(df)
                return_tables[race_id] = df
                time.sleep(1)
            except IndexError:
                continue
            except Exception as e:
                print(e)
                break
            except:
                break
        
        # pd.DataFrame型にして一つのデータにまとめる
        return_tables_df = pd.concat([return_tables[key] for key in return_tables])
        
        return return_tables_df
    
    # 複勝の勝ち馬と払い戻しのデータを取り出す
    @property
    def fukusho(self):
        fukusho = self.return_tables[self.return_tables[0] == '複勝'][[1, 2]]
        
        # 勝ち馬
        wins = fukusho[1].str.split('br', expand=True)[[0, 1, 2]]
        wins.columns = ['win_0', 'win_1', 'win_2']
        
        # 払い戻し
        returns = fukusho[2].str.split('br', expand=True)[[0, 1, 2]]
        returns.columns = ['return_0', 'return_1', 'return_2']
        
        df = pd.concat([wins, returns], axis=1)
        
        # int型に変換できないデータがあるので処理
        for column in df.columns:
            df[column] = df[column].str.replace(',', '')
            
        df = df.fillna(0).astype(int)
        
        return df
    
    # 単勝の勝ち馬と払い戻しのデータを取り出す
    @property
    def tansho(self):
        tansho = self.return_tables[self.return_tables[0] == '単勝'][[1, 2]]
        tansho.columns = ['win', 'return']
        
        for column in tansho.columns:
            tansho[column] = pd.to_numeric(tansho[column], errors='coerce')
            
        return tansho
    
    # 馬連の勝ち馬と払い戻しのデータを取り出す
    @property
    def umaren(self):
        umaren = self.return_tables[self.return_tables[0] == '馬連'][[1, 2]]
        wins = umaren[1].str.split('-', expand=True)[[0, 1]].add_prefix('win_')
        return_ = umaren[2].rename('return')
        df = pd.concat([wins, return_], axis=1)      
        df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
        
        return df

# 予測モデルを評価するクラス
class ModelEvaluator:
    def __init__(self, model, return_tables_path_list):
        self.model = model
        self.rt = Return.read_pickle(return_tables_path_list)
        self.fukusho = self.rt.fukusho
        self.tansho = self.rt.tansho
        self.umaren = self.rt.umaren
    
    # 1（3着以内）になる確率を予測
    def predict_proba(self, x, std=True, minmax=False):
        proba = pd.Series(self.model.predict_proba(x.drop(['単勝'], axis=1))[:, 1], index=x.index)
        if std:
            # 標準化：レース内で相対評価する
            standard_scaler = lambda x: (x - x.mean()) / x.std()
            proba = proba.groupby(level=0).transform(standard_scaler)
        if minmax:
            # MinMaxスケーリング：データを0から1にする
            proba = (proba - proba.min()) / (proba.max() - proba.min())
        return proba
    
    # thresholdの値によってpredict_probaで出した値を0か1に決定する
    def predict(self, x, threshold=0.5):
        y_pred = self.predict_proba(x)
        return [0 if p < threshold else 1 for p in y_pred]
    
    # スコアを表示
    def score(self, y_true, x):
        return roc_auc_score(y_true, self.predict_proba(x))
    
    # 変数の重要度を出力
    def feature_importance(self, x, n_display=20):
        importances = pd.DataFrame(
            {"features": x.columns, "importance": self.model.feature_importances_}
        )
        return importances.sort_values('importance', ascending=False)[:n_display]
    
    # 予測したレースIDと馬番を出力する
    def pred_table(self, x, threshold=0.5, bet_only=True):
        pred_table = x.copy()[['馬番', '単勝']]
        pred_table['pred'] = self.predict(x, threshold)
        
        if bet_only:
            return pred_table[pred_table['pred'] == 1][['馬番', '単勝']]
        
        return pred_table
    
    # 複勝の払い戻しを表示
    def fukusho_return(self, x, threshold=0.5):
        pred_table = self.pred_table(x, threshold)
        n_bets = len(pred_table)
        money = -100 * n_bets
        df = self.fukusho.copy()
        df = df.merge(pred_table, left_index=True, right_index=True, how='right')
        for i in range(3):
            money += df[df[f'win_{i}'] == df['馬番']][f'return_{i}'].sum()
        return_rate = (n_bets * 100 + money) / (n_bets * 100)
        return n_bets, return_rate
    
    # 単勝の払い戻しを表示
    def tansho_return(self, x, threshold=0.5):
        pred_table = self.pred_table(x, threshold)
        n_bets = len(pred_table)
        n_races = pred_table.index.nunique()
        
        money = -100 * n_bets
        df = self.tansho.copy()
        df = df.merge(pred_table, left_index=True, right_index=True, how='right')
        
        std = ((df['win'] == df['馬番']) * df['return']).groupby(level=0).sum().std() * np.sqrt(n_races) / (100 * n_bets)
        
        n_hits = len(df[df['win'] == df['馬番']])
        money += df[df['win'] == df['馬番']]['return'].sum()
        return_rate = (n_bets * 100 + money) / (n_bets * 100)
        return n_bets, return_rate, n_hits, std
    
    # モデルによって「賭ける」と判断された馬たち
    def tansho_return_proper(self, x, threshold=0.5):
        pred_table = self.pred_table(x, threshold)
        n_bets = len(pred_table)
        n_races = pred_table.index.nunique()
        
        # 払い戻し表にpred_tableをマージ
        df = self.tansho.copy()
        df = df.merge(pred_table, left_index=True, right_index=True, how='right')
        
        bet_money = (1 / pred_table['単勝']).sum()
        
        std = ((df['win'] == df['馬番']).astype(int)).groupby(level=0).sum().std() * np.sqrt(n_races) / bet_money
        
        # 単勝適正回収値を計算
        n_hits = len(df.query('win == 馬番'))
        return_rate = n_hits / bet_money
        
        return n_bets, return_rate, n_hits, std
    
    # 馬連の払い戻しを表示
    def umaren_return(self, x, threshold=0.5):
        pred_table = self.pred_table(x, threshold)
        hit = {}
        n_bets = 0
        for race_id, preds in pred_table.groupby(level=0):
            n_bets += comb(len(preds, 2))
            hit[race_id] = set(self.umaren.loc[race_id][['win_0', 'win_1']]).issubset(set(preds))
        return_rate = self.umaren.index.map(hit).values * self.umaren['return'].sum() / (n_bets * 100)
        return n_bets, return_rate

#### その他関数等定義

In [3]:
# 時系列に沿って訓練データとテストデータに分ける関数
def split_data(df, test_size=0.3):
    sorted_id_list = df.sort_values('date').index.unique()   
    train_id_list = sorted_id_list[:round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)):]
    train = df.loc[train_id_list]
    test = df.loc[test_id_list]
    return train, test

# 回収率を計算する関数
def gain(return_func, x, n_samples=100, t_range=[0.5, 3.5]):
    gain = {}
    for i in tqdm(range(n_samples)):
        # min_thresholdから1まで、n_samples等分して、thresholdをfor文で回す
        threshold = t_range[1] * i / n_samples + t_range[0] * (1 - (i / n_samples))
        n_bets, return_rate, n_hits, std = return_func(x, threshold)
        if n_bets > 2:
            gain[threshold] = {'return_rate': return_rate, 'n_hits': n_hits, 'std': std, 'n_bets': n_bets}
    return pd.DataFrame(gain).T

# 重複のないデータを作成する関数
def update_data(old, new):
    filtered_old = old[~old.index.isin(new.index)]
    return pd.concat([filtered_old, new])

# 標準偏差付き回収率プロット
def plot(df, label=' '):
    # 標準偏差で幅をつけて薄くプロット
    plt.fill_between(df.index, y1=df['return_rate']-df['std'], y2=df['return_rate']+df['std'], alpha=0.3)
    # 回収率で実践をプロット
    plt.plot(df.index, df['return_rate'], label=label)
    plt.legend()
    plt.grid(True)

# 開催場所をidに変換するための辞書型
place_dict = {
    '札幌': '01', '函館': '02', '福島': '03', '新潟': '04', '東京': '05',
    '中山': '06', '中京': '07', '京都': '08', '阪神': '09', '小倉': '10'
}

# レースタイプをレース結果データと整合させるための辞書型
race_type_dict = {
    '芝': '芝', 'ダ': 'ダート', '障': '障害'
}

#### 2017年～2021年までのレース結果データをスクレイピング

In [4]:
# 2017年のレースIDリスト
# race_id_list_2017 = []
# for place in range(1, 11):
#     for kai in range(1, 6):
#         for day in range(1, 13):
#             for r in range(1, 13):
#                 race_id = f'2017{str(place).zfill(2)}{str(kai).zfill(2)}{str(day).zfill(2)}{str(r).zfill(2)}'
#                 race_id_list_2017.append(race_id)

# 2018年のレースIDリスト
# race_id_list_2018 = []
# for place in range(1, 11):
#     for kai in range(1, 6):
#         for day in range(1, 13):
#             for r in range(1, 13):
#                 race_id = f'2018{str(place).zfill(2)}{str(kai).zfill(2)}{str(day).zfill(2)}{str(r).zfill(2)}'
#                 race_id_list_2018.append(race_id)

# 2019年のレースIDリスト
# race_id_list_2019 = []
# for place in range(1, 11):
#     for kai in range(1, 6):
#         for day in range(1, 13):
#             for r in range(1, 13):
#                 race_id = f'2019{str(place).zfill(2)}{str(kai).zfill(2)}{str(day).zfill(2)}{str(r).zfill(2)}'
#                 race_id_list_2019.append(race_id)

# 2020年のレースIDリスト
# race_id_list_2020 = []
# for place in range(1, 11):
#     for kai in range(1, 6):
#         for day in range(1, 13):
#             for r in range(1, 13):
#                 race_id = f'2020{str(place).zfill(2)}{str(kai).zfill(2)}{str(day).zfill(2)}{str(r).zfill(2)}'
#                 race_id_list_2020.append(race_id)

# 2021年のレースIDリスト
# race_id_list_2021 = []
# for place in range(1, 11):
#     for kai in range(1, 6):
#         for day in range(1, 13):
#             for r in range(1, 13):
#                 race_id = f'2021{str(place).zfill(2)}{str(kai).zfill(2)}{str(day).zfill(2)}{str(r).zfill(2)}'
#                 race_id_list_2021.append(race_id)

In [5]:
# スクレイピングしたのでコメント

# 2017年のレース結果データ
# results_2017 = Results.scrape(race_id_list)
# results_2017.to_pickle('pickle/results_2017.pickle')
# results_2017 = pd.read_pickle('pickle/results_2017.pickle')

# 2018年のレース結果データ
# results_2018 = Results.scrape(race_id_list_2018)
# results_2018.to_pickle('pickle/results_2018.pickle')
# results_2018 = pd.read_pickle('pickle/results_2018.pickle')

# 2019年のレース結果データ
# results_2019 = Results.scrape(race_id_list_2019)
# results_2019.to_pickle('pickle/results_2019.pickle')
# results_2019 = pd.read_pickle('pickle/results_2019.pickle')

# 2020年のレース結果データ
# results_2020 = Results.scrape(race_id_list_2020)
# results_2020.to_pickle('pickle/results_2020.pickle')
# results_2020 = pd.read_pickle('pickle/results_2020.pickle')

# 2021年のレース結果データ
# results_2021 = Results.scrape(race_id_list_2021)
# results_2021.to_pickle('pickle/results_2021.pickle')
# results_2021 = pd.read_pickle('pickle/results_2021.pickle')

# レース結果データの結合
# r = Results.read_pickle(['pickle/results_2017.pickle', 'pickle/results_2018.pickle', 'pickle/results_2019.pickle', 'pickle/results_2020.pickle', 'pickle/results_2021.pickle'])

# 前処理
# r.preprocessing()

#### 2017～2021年までの馬の過去成績データをスクレイピング

In [6]:
# 2017年の馬IDリスト
# horse_id_list_2017 = results_2017['horse_id'].unique()

# 2018年の馬IDリスト
# horse_id_list_2018 = results_2018['horse_id'].unique()

# 2019年の馬IDリスト
# horse_id_list_2019 = results_2019['horse_id'].unique()

# 2020年の馬IDリスト
# horse_id_list_2020 = results_2020['horse_id'].unique()

# 2021年の馬IDリスト
# horse_id_list_2021 = results_2021['horse_id'].unique()

In [7]:
# スクレイピングして結合したのでコメント

# 2017年の馬の過去成績データ
# horse_results_2017 = HorseResults.scrape(horse_id_list_2017)
# horse_results_2017.to_pickle('pickle/horse_results_2017.pickle')

# 2018年の馬の過去成績データ
# horse_results_2018 = HorseResults.scrape(horse_id_list_2018)
# horse_results_2018.to_pickle('pickle/horse_results_2018.pikcle')

# 2019年の馬の過去成績データ
# horse_results_2019 = HorseResults.scrape(horse_id_list_2019)
# horse_results_2019.to_pickle('pickle/horse_results_2019.pikcle')

# 2020年の馬の過去成績データ
# horse_results_2020 = HorseResults.scrape(horse_id_list_2020)
# horse_results_2020.to_pickle('pickle/horse_results_2020.pikcle')

# 2021年の馬の過去成績データ
# horse_results_2021 = HorseResults.scrape(horse_id_list_2021)
# horse_results_2021.to_pickle('pickle/horse_results_2021.pickle')

# 馬の過去成績データの結合
# hr = HorseResults.read_pickle(['pickle/horse_results_2017.pickle', 'pickle/horse_results_2018.pickle', 'pickle/horse_results_2019.pickle', 'pickle/horse_results_2020.pickle', 'pickle/horse_results_2021.pickle'])

# レース結果データに馬の過去成績データを追加
# r.merge_horse_results(hr)

#### 2017年～2021年までの血統データをスクレイピング

In [8]:
# スクレイピングして結合したのでコメント

# 2017年の血統データ
# peds_2017 = Peds.scrape(horse_id_list_2017)
# peds_2017.to_pickle('pickle/peds_2017.pickle')

# 2018年の血統データ
# peds_2018 = Peds.scrape(horse_id_list_2018)
# peds_2018.to_pickle('pickle/peds_2018.pickle')

# 2019年の血統データ
# peds_2019 = Peds.scrape(horse_id_list_2019)
# peds_2019.to_pickle('pickle/peds_2019.pickle')

# 2020年の血統データ
# peds_2020 = Peds.scrape(horse_id_list_2020)
# peds_2020.to_pickle('pickle/peds_2020.pickle')

# 2021年の血統データ
# peds_2021 = Peds.scrape(horse_id_list_2021)
# peds_2021.to_pickle('pickle/peds_2021.pickle')

# 血統データの結合
# p = Peds.read_pickle(['pickle/peds_2017.pickle', 'pickle/peds_2018.pickle', 'pickle/peds_2019.pickle', 'pickle/peds_2020.pickle', 'pickle/peds_2021.pickle'])

# レース結果データに5世代分の血統データを追加
# p.encode()
# r.merge_peds(p.peds_e)

# カテゴリ変数の処理
# r.process_categorical()

# pickleファイルに保存
# r.data_c.to_pickle('pickle/results_all_c.pickle')

In [9]:
results_c = pd.read_pickle('pickle/results_all_c.pickle')

#### 2017年～2021年の払い戻し表データをスクレイピング

In [10]:
# レース結果データから取得した2017年のレースIDリスト
# race_id_list_2017 = results_2017.index.unique()

# レース結果データから取得した2018年のレースIDリスト
# race_id_list_2018 = results_2018.index.unique()

# レース結果データから取得した2019年のレースIDリスト
# race_id_list_2019 = results_2019.index.unique()

# レース結果データから取得した2020年のレースIDリスト
# race_id_list_2020 = results_2020.index.unique()

# レース結果データから取得した2021年のレースIDリスト
# race_id_list_2021 = results_2021.index.unique()

In [11]:
# スクレイピングして結合したのでコメント

# 2017年の払い戻し表データ
# return_tables_2017 = Return.scrape(race_id_list_2017)
# return_tables_2017.to_pickle('pickle/return_tables_2017.pickle')

# 2018年の払い戻し表データ
# return_tables_2018 = Return.scrape(race_id_list_2018)
# return_tables_2018.to_pickle('pickle/return_tables_2018.pickle')

# 2019年の払い戻し表データ
# return_tables_2019 = Return.scrape(race_id_list_2019)
# return_tables_2019.to_pickle('pickle/return_tables_2019.pickle')

# 2020年の払い戻し表データ
# return_tables_2020 = Return.scrape(race_id_list_2020)
# return_tables_2020.to_pickle('pickle/return_tables_2020.pickle')

# 2021年の払い戻し表データ
# return_tables_2021 = Return.scrape(race_id_list_2021)
# return_tables_2021.to_pickle('pickle/return_tables_2021.pickle')

# 払い戻し表データの結合
# return_tables = Return.read_pickle(['pickle/return_tables_2017.pickle', 'pickle/return_tables_2018.pickle', 'pickle/return_tables_2019.pickle', 'pickle/return_tables_2020.pickle', 'pickle/return_tables_2021.pickle'])

# pickleファイルに保存
# return_tables.return_tables.to_pickle('pickle/return_tables.pickle')

In [12]:
return_tables = pd.read_pickle('pickle/return_tables.pickle')

#### 学習

In [13]:
# 時系列に沿って訓練データとテストデータに分ける
train, test = split_data(results_c)
X_train = train.drop(['rank', 'date', '単勝'], axis=1)
y_train = train['rank']
X_test = test.drop(['rank', 'date'], axis=1)
y_test = test['rank']

In [14]:
# optunaでパラメータチューニングしたパラメータ
params = {
 'objective': 'binary',
 'random_state': 100,
 'feature_pre_filter': False,
 'lambda_l1': 1.3132588653273114e-05,
 'lambda_l2': 7.718638255265974,
 'num_leaves': 33,
 'feature_fraction': 1.0,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20
}

# LGBMで学習
lgb_clf = lgb.LGBMClassifier(**params)
lgb_clf.fit(X_train.values, y_train.values)

LGBMClassifier(bagging_fraction=1.0, bagging_freq=0, feature_fraction=1.0,
               feature_pre_filter=False, lambda_l1=1.3132588653273114e-05,
               lambda_l2=7.718638255265974, num_leaves=33, objective='binary',
               random_state=100)

In [15]:
# ModelEvaluatorクラスのオブジェクトを作成
me = ModelEvaluator(lgb_clf, ['pickle/return_tables.pickle'])

In [ ]:
g_tansho = gain(me.tansho_return, X_test)

 98%|█████████▊| 98/100 [07:14<00:09,  4.64s/it]

In [ ]:
plot(g_tansho, 'tansho')